In [1]:
%pip install spuco

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

In [12]:
import torch
device = torch.device("cpu")

In [35]:
# To ensure uniformity
from spuco.utils import set_seed
set_seed(0)

In [8]:
from spuco.robust_train import ERM
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]  # Chosen based on example notebooks
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train",
    label_noise=0.001
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

valset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val"
)
valset.initialize()



100%|██████████| 10000/10000 [00:01<00:00, 6502.85it/s]


In [42]:
# Example data
T.ToPILImage()(trainset[0][0]).resize((28,28))

### Training ERM

In [43]:
from spuco.models import model_factory
from torch.optim import SGD


model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)

erm = ERM(
    model=model,
    num_epochs=2,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [44]:
erm.train()

Epoch 1: 100%|██████████| 751/751 [00:32<00:00, 22.79batch/s, accuracy=100.0%, loss=0.00382]


In [45]:
from spuco.evaluate import Evaluator

evaluator_ERM = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator_ERM.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:18,  1.33it/s]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:01<00:15,  1.51it/s]

Group (0, 1) Accuracy: 0.0


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:02<00:14,  1.52it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:17<00:00,  1.58it/s]

Group (4, 3) Accuracy: 0.0


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:18<00:00,  1.34it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 100.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 100.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 100.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 100.0,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 100.0}

In [49]:
evaluator_ERM.worst_group_accuracy

((0, 1), 0.0)

In [50]:
evaluator_ERM.average_accuracy

99.39380051662361

In [51]:
evaluator_ERM.evaluate_spurious_attribute_prediction()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


100.0

## Clustering inputs


In [52]:
from spuco.group_inference import Cluster, ClusterAlg

logits = erm.trainer.get_trainset_outputs()
cluster = Cluster(
    Z=logits,
    class_labels=trainset.labels,
    cluster_alg=ClusterAlg.KMEANS,
    num_clusters=2,
    device=device,
    verbose=True
)
group_partition = cluster.infer_groups()

Clustering class-wise: 100%|██████████| 5/5 [00:00<00:00, 12.85it/s]


In [53]:
for key in sorted(group_partition.keys()):
    print(key, len(group_partition[key]))

(0, 0) 10073
(0, 1) 61
(1, 0) 9615
(1, 1) 56
(2, 0) 8956
(2, 1) 56
(3, 0) 9689
(3, 1) 61
(4, 0) 9380
(4, 1) 57


In [54]:
evaluator_Cluster = Evaluator(
    testset=trainset,
    group_partition=group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator_Cluster.evaluate()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Evaluating group-wise accuracy:  10%|█         | 1/10 [00:03<00:35,  3.98s/it]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:  20%|██        | 2/10 [00:04<00:16,  2.00s/it]

Group (0, 1) Accuracy: 0.0


Evaluating group-wise accuracy:  30%|███       | 3/10 [00:07<00:15,  2.21s/it]

...

Evaluating group-wise accuracy:  90%|█████████ | 9/10 [00:16<00:01,  1.94s/it]

Group (4, 0) Accuracy: 100.0


Evaluating group-wise accuracy: 100%|██████████| 10/10 [00:17<00:00,  1.74s/it]

Group (4, 1) Accuracy: 0.0


{(0, 0): 100.0,
 (0, 1): 0.0,
 (1, 0): 100.0,
 (1, 1): 0.0,
 (2, 0): 100.0,
 (2, 1): 0.0,
 (3, 0): 100.0,
 (3, 1): 0.0,
 (4, 0): 100.0,
 (4, 1): 0.0}

## Group-Balanced ERM training

In [55]:
from torch.optim import SGD
from spuco.robust_train import GroupBalanceBatchERM, ClassBalanceBatchERM
from spuco.models import model_factory

model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
group_balance_erm = GroupBalanceBatchERM(
    model=model,
    num_epochs=5,
    trainset=trainset,
    group_partition=group_partition,
    batch_size=64,
    optimizer=torch.optim.SGD(model.parameters(), lr=1e-2, weight_decay=5e-4, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
group_balance_erm.train()

Epoch 4: 100%|██████████| 751/751 [00:23<00:00, 32.42batch/s, accuracy=100.0%, loss=0.00017]


In [56]:
evaluator_GroupBalanace = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator_GroupBalanace.evaluate()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:20,  1.15it/s]

Group (0, 0) Accuracy: 99.76359338061465


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:01<00:16,  1.36it/s]

Group (0, 1) Accuracy: 73.5224586288416


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:02<00:16,  1.36it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:17<00:00,  1.36it/s]

Group (4, 3) Accuracy: 69.1919191919192


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:18<00:00,  1.35it/s]

Group (4, 4) Accuracy: 99.4949494949495


{(0, 0): 99.76359338061465,
 (0, 1): 73.5224586288416,
 (0, 2): 71.63120567375887,
 (0, 3): 78.95981087470449,
 (0, 4): 71.39479905437352,
 (1, 0): 76.28361858190709,
 (1, 1): 98.77750611246944,
 (1, 2): 66.17647058823529,
 (1, 3): 63.48039215686274,
 (1, 4): 61.27450980392157,
 (2, 0): 82.13333333333334,
 (2, 1): 70.66666666666667,
 (2, 2): 100.0,
 (2, 3): 82.93333333333334,
 (2, 4): 41.44385026737968,
 (3, 0): 69.09547738693468,
 (3, 1): 62.21662468513854,
 (3, 2): 56.92695214105793,
 (3, 3): 98.99244332493703,
 (3, 4): 70.7808564231738,
 (4, 0): 78.08564231738035,
 (4, 1): 29.974811083123427,
 (4, 2): 27.455919395465994,
 (4, 3): 69.1919191919192,
 (4, 4): 99.4949494949495}

In [57]:
evaluator_GroupBalanace.worst_group_accuracy

((4, 2), 27.455919395465994)

In [58]:
evaluator_GroupBalanace.average_accuracy

99.19735392043009

In [59]:
evaluator_GroupBalanace.evaluate_spurious_attribute_prediction()

41.63